## Background Removal with Robust PCA

### Parte 1: Execução do tutorial exemplo

#### Getting Started

In [ ]:
import requests, zipfile, os
url = "https://download1337.mediafire.com/42s21eo7zgjg/rkxwc8t203b1m2m/data.zip"
filename = "data.zip"
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)
zip_ref = zipfile.ZipFile(filename)
zip_ref.extractall("./")
zip_ref.close()
os.remove(filename)

In [ ]:
# Import needed libraries
import moviepy.editor as mpe
import numpy as np
import scipy
from PIL import Image

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
video = mpe.VideoFileClip("PETS09-S2L2.avi")

In [ ]:
video.subclip(0,50).ipython_display(width=300)

In [ ]:
video.duration

In [ ]:
scale = 25
dims = (int(240 * (scale / 100)), int(320 * (scale / 100)))

#### Helper Methods

In [ ]:
def create_data_matrix_from_video(clip, k=5, scale=50):
  new_scale = (int(320 * (scale / 100)), int(240 * (scale / 100)))
  return np.vstack([np.array(Image.fromarray(rgb2gray(clip.get_frame(i / float(k))).astype(np.uint8)).resize(size=new_scale)).flatten() for i in range(k * int(clip.duration))]).T

In [ ]:
def rgb2gray(rgb):
  return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [ ]:
def plt_images(M, A, E, index_array, dims, filename=None):
    f = plt.figure(figsize=(15, 10))
    r = len(index_array)
    # pics = r * 3
    for k, i in enumerate(index_array):
      for j, mat in enumerate([M, A, E]):
        sp = f.add_subplot(r, 3, 3*k + j + 1)
        sp.axis('Off')
        pixels = mat[:,i]
        if isinstance(pixels, scipy.sparse.csr_matrix):
            pixels = pixels.todense()
        plt.imshow(np.reshape(pixels, dims), cmap='gray')
    return f

In [ ]:
def plots(ims, dims, figsize=(15,20), rows=1, interp=False, titles=None):
  if type(ims[0]) is np.ndarray:
    ims = np.array(ims)
  f = plt.figure(figsize=figsize)
  for i in range(len(ims)):
    sp = f.add_subplot(rows, len(ims)//rows, i+1)
    sp.axis('Off')
    plt.imshow(np.reshape(ims[i], dims), cmap="gray")

#### Load and view the data

In [ ]:
M = create_data_matrix_from_video(video, 100, scale)

In [ ]:
print(dims, M.shape)

In [ ]:
plt.imshow(np.reshape(M[:, 140], dims), cmap='gray')

In [ ]:
np.save("low_PETS09-S2L2_matrix.npy", M)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(M, cmap='gray')

In [ ]:
plt.imsave(fname="image1.jpg", arr=np.reshape(M[:,140], dims), cmap='gray')

#### SVD

In [ ]:
from sklearn import decomposition

In [ ]:
u, s, v = decomposition.randomized_svd(M, 2)

In [ ]:
u.shape, s.shape, v.shape

In [ ]:
low_rank = u @ np.diag(s) @ v

In [ ]:
low_rank.shape

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(low_rank, cmap='gray')

In [ ]:
plt.imshow(np.reshape(low_rank[:,140], dims), cmap='gray')

In [ ]:
plt.imshow(np.reshape(M[:,550] - low_rank[:,550], dims), cmap='gray');

In [ ]:
u, s, v = decomposition.randomized_svd(M, 1)

In [ ]:
u.shape, s.shape, v.shape

In [ ]:
low_rank = u @ np.diag(s) @ v

In [ ]:
low_rank.shape

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(low_rank, cmap='gray')

In [ ]:
plt.imshow(np.reshape(M[:,550] - low_rank[:,550], dims), cmap='gray')

In [ ]:
plt.imshow(np.reshape(M[:,140] - low_rank[:,140], dims), cmap='gray')

In [ ]:
plt.imshow(np.reshape(M[:,140] - low_rank[:,140], dims)[10:45, 23:70], cmap='gray')

#### Principal Component Analysis (PCA)

In [ ]:
from scipy import sparse
from sklearn.utils.extmath import randomized_svd
import fbpca

In [ ]:
TOL=1e-9
MAX_ITERS=3

In [ ]:
def converged(Z, d_norm):
    err = np.linalg.norm(Z, 'fro') / d_norm
    print('error: ', err)
    return err < TOL

In [ ]:
def shrink(M, tau):
    S = np.abs(M) - tau
    return np.sign(M) * np.where(S>0, S, 0)

In [ ]:
def _svd(M, rank): return fbpca.pca(M, k=min(rank, np.min(M.shape)), raw=True)

In [ ]:
def norm_op(M): return _svd(M, 1)[1][0]

In [ ]:
def svd_reconstruct(M, rank, min_sv):
    u, s, v = _svd(M, rank)
    s -= min_sv
    nnz = (s > 0).sum()
    return u[:,:nnz] @ np.diag(s[:nnz]) @ v[:nnz], nnz

In [ ]:
def pcp(X, maxiter=10, k=10): # refactored
    m, n = X.shape
    trans = m<n
    if trans: X = X.T; m, n = X.shape
        
    lamda = 1/np.sqrt(m)
    op_norm = norm_op(X)
    Y = np.copy(X) / max(op_norm, np.linalg.norm( X, np.inf) / lamda)
    mu = k*1.25/op_norm; mu_bar = mu * 1e7; rho = k * 1.5
    
    d_norm = np.linalg.norm(X, 'fro')
    L = np.zeros_like(X); sv = 1
    
    examples = []
    
    for i in range(maxiter):
        print("rank sv:", sv)
        X2 = X + Y/mu
        
        # update estimate of Sparse Matrix by "shrinking/truncating": original - low-rank
        S = shrink(X2 - L, lamda/mu)
        
        # update estimate of Low-rank Matrix by doing truncated SVD of rank sv & reconstructing.
        # count of singular values > 1/mu is returned as svp
        L, svp = svd_reconstruct(X2 - S, sv, 1/mu)
        
        # If svp < sv, you are already calculating enough singular values.
        # If not, add 20% (in this case 240) to sv
        sv = svp + (1 if svp < sv else round(0.05*n))
        
        # residual
        Z = X - L - S
        Y += mu*Z; mu *= rho
        
        examples.extend([S[140,:], L[140,:]])
        
        if m > mu_bar: m = mu_bar
        if converged(Z, d_norm): break
    
    if trans: L=L.T; S=S.T
    return L, S, examples

In [ ]:
m, n = M.shape
round(m * .05)

In [ ]:
L, S, examples = pcp(M, maxiter=5, k=10)

In [ ]:
plots(examples, dims, rows=5)

In [ ]:
f = plt_images(M, S, L, [140], dims)

In [ ]:
np.save("high_res_L.npy", L)
np.save("high_res_S.npy", S)

In [ ]:
f = plt_images(M, S, L, [0, 100, 1000], dims)

### Parte 2: Aplicação em Problema Real

#### Exemplo 1

In [ ]:
real_video_1 = mpe.VideoFileClip("TUD-Crossing.avi")

In [ ]:
M_1 = create_data_matrix_from_video(real_video_1, 100, scale)

In [ ]:
print(dims, M_1.shape)

In [ ]:
plt.imshow(np.reshape(M_1[:, 140], dims), cmap='gray')

In [ ]:
np.save("low_TUD-Crossing_matrix.npy", M_1)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(M_1, cmap='gray')

In [ ]:
plt.imsave(fname="image2.jpg", arr=np.reshape(M_1[:,140], dims), cmap='gray')

In [ ]:
u_1, s_1, v_1 = decomposition.randomized_svd(M_1, 2)

In [ ]:
u_1.shape, s_1.shape, v_1.shape

In [ ]:
low_rank_1 = u_1 @ np.diag(s_1) @ v_1
low_rank_1.shape

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(low_rank_1, cmap='gray')

In [ ]:
plt.imshow(np.reshape(low_rank_1[:,140], dims), cmap='gray')

In [ ]:
plt.imshow(np.reshape(M_1[:,550] - low_rank_1[:,550], dims), cmap='gray')

In [ ]:
u_1, s_1, v_1 = decomposition.randomized_svd(M_1, 1)

In [ ]:
u_1.shape, s_1.shape, v_1.shape


In [ ]:
low_rank_1 = u_1 @ np.diag(s_1) @ v_1
low_rank_1.shape

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(low_rank_1, cmap='gray')

In [ ]:
plt.imshow(np.reshape(M_1[:,550] - low_rank_1[:,550], dims), cmap='gray')


In [ ]:
plt.imshow(np.reshape(M_1[:,140] - low_rank_1[:,140], dims), cmap='gray')


In [ ]:
plt.imshow(np.reshape(M_1[:,140] - low_rank_1[:,140], dims)[10:45, 23:70], cmap='gray')

In [ ]:
m, n = M_1.shape
round(m * .05)

In [ ]:
L, S, examples = pcp(M_1, maxiter=5, k=10)

In [ ]:
f = plt_images(M_1, S, L, [140], dims)

In [ ]:
f = plt_images(M_1, S, L, [140], dims)

In [ ]:
np.save("high_res_L1.npy", L)
np.save("high_res_S1.npy", S)

In [ ]:
f = plt_images(M_1, S, L, [0, 100, 1000], dims)

#### Exemplo 2

In [ ]:
real_video_2 = mpe.VideoFileClip("AVG-TownCentre.avi")

In [ ]:
M_2 = create_data_matrix_from_video(real_video_2, 100, scale)

In [ ]:
print(dims, M_2.shape)

In [ ]:
plt.imshow(np.reshape(M_2[:, 140], dims), cmap='gray')

In [ ]:
np.save("low_TUD-Crossing_matrix.npy", M_2)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(M_2, cmap='gray')

In [ ]:
plt.imsave(fname="image3.jpg", arr=np.reshape(M_2[:,140], dims), cmap='gray')

In [ ]:
u, s, v = decomposition.randomized_svd(M_2, 2)

In [ ]:
u.shape, s.shape, v.shape

In [ ]:
low_rank = u @ np.diag(s) @ v
low_rank.shape

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(low_rank, cmap='gray')

In [ ]:
plt.imshow(np.reshape(low_rank[:,140], dims), cmap='gray')

In [ ]:
plt.imshow(np.reshape(M_2[:,550] - low_rank[:,550], dims), cmap='gray')

In [ ]:
u, s, v = decomposition.randomized_svd(M_2, 1)

In [ ]:
u.shape, s.shape, v.shape


In [ ]:
low_rank = u @ np.diag(s) @ v
low_rank.shape

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(low_rank, cmap='gray')

In [ ]:
plt.imshow(np.reshape(M_2[:,550] - low_rank[:,550], dims), cmap='gray')


In [ ]:
plt.imshow(np.reshape(M_2[:,140] - low_rank[:,140], dims), cmap='gray')


In [ ]:
plt.imshow(np.reshape(M_2[:,140] - low_rank[:,140], dims)[10:45, 23:70], cmap='gray')

In [ ]:
m, n = M_2.shape
round(m * .05)

In [ ]:
L, S, examples = pcp(M_2, maxiter=5, k=10)

In [ ]:
f = plt_images(M_2, S, L, [140], dims)

In [ ]:
f = plt_images(M_2, S, L, [140], dims)

In [ ]:
np.save("high_res_L2.npy", L)
np.save("high_res_S2.npy", S)

In [ ]:
f = plt_images(M_2, S, L, [0, 100, 1000], dims)